In [1]:
import numpy as np
import nibabel as nb
from scipy.spatial.transform import Rotation as rotation

In [2]:
# calc_transforms():

# Given
#   - A nibable img as returned by nb.load()
# Compute
#   - The vox2ras matrix (this is simply img.affine)
#   - The grad2ras matrix (mri_info calls this xform info)
#   - See mri.cpp:12818
def calc_transforms(img):
    vox2ras = np.copy(img.affine)
    grad2ras = np.copy(vox2ras)

    # - pixdims is same as xsize, ysize, zsize in mri.cpp:12818 
    # - c is same as c_r, c_a, c_s in mri.cpp:12818
    # - use of 1:4 here is probably fragile
    pixdims = img.header['pixdim'][1:4]
    c = img.header['dim'][1:4] / 2

    grad2ras[0:3,0:3] = vox2ras[0:3,0:3] / pixdims
    grad2ras[0:3,3]   = vox2ras[0:3,3]   + vox2ras[0:3,0:3] @ c

    return vox2ras, grad2ras

In [3]:
# calc_aa():

# Given:
#  - An image taken with a particular FOV: which gives
#    - V2R: A VOX2RAS matrix (4x4)
#    - G2R: A GRAD2RAS matrix (4x4; xform info)
#  - An alignment vector along 0th, 1st or 2nd dimension
#    - align:
#    - polarity: 1 or -1
#  - 2 voxel coordinates of that image;
#    - p1_vox: where you'd like the origin for the new FOV to be (4x1)
#    - p2_vox: a point you would like alinged with sag/cor/tra (4x1)
#    
# Compute:
#  - A transformation that when applied to G2R (G2R * T or T * G2R?) results in
#    - T_ras2ras: A RAS2RAS trasform (4x4), such that
#      - T * p1_ras = [0,0,0,1]
#      - T * p2_ras is parrallel to a_ras 
#    - T_lps2lps: Same transform, but in lps2lps
#
# We know
#  - R2L: A RAS to LPS conversion matrix
#  - We can convert points or vectors:
#    - p_ras = VOX2RAS * p_vox
#    - p_vox = inv(VOX2RAS) * p_ras
#  - There are infite translations T that exits, we aribtrarily pick one
#    - There's probably a smarter way to constrain to get a unique solution

def calc_aa(img_filename, p1_vox4, p2_vox4, align_dim=0, polarity=1):

    # converts from ras2lps and vice versa (note that R2L == inv(R2L) )
    R2L            = np.array([[-1,  0, 0, 0],
                               [ 0, -1, 0, 0],
                               [ 0,  0, 1, 0],
                               [ 0,  0, 0, 1]], dtype=np.float64)
    polarity_flip3 = np.array([[-1,  0, 0],
                               [ 0, -1, 0],
                               [ 0,  0, -1]], dtype=np.float64)

    img = nb.load(img_filename)
    V2R, G2R = calc_transforms(img)
    
    p1_ras4 = V2R @ p1_vox4
    p2_ras4 = V2R @ p2_vox4

    p1_ras3 = p1_ras4[0:3]
    p2_ras3 = p2_ras4[0:3]
    
    # The primary vector to align with
    q1_ras3 = (p2_ras3 - p1_ras3) / np.linalg.norm(p2_ras3 - p1_ras3)
    if polarity < 0:
        q1_ras3 = polarity_flip3 @ q1_ras3
    
    # Find other 2 arbitrary (but deterministic) ortho vectors
    # to build our space
    # This wont work if the vector we cross with is the zero vector,
    # so choose the point with the larger norm as the arb vector
    if (np.linalg.norm(p1_ras3) > np.linalg.norm(p2_ras3)):
        q2_ras3 = np.cross(q1_ras3.T, p1_ras3.T).T / np.linalg.norm(np.cross(q1_ras3.T, p1_ras3.T).T)
    else:
        q2_ras3 = np.cross(q1_ras3.T, p2_ras3.T).T / np.linalg.norm(np.cross(q1_ras3.T, p2_ras3.T).T)
    q3_ras3 = np.cross(q1_ras3.T, q2_ras3.T).T
    
    if align_dim == 0:
        rot = np.linalg.inv(np.hstack((q1_ras3,q2_ras3,q3_ras3)))
    elif align_dim == 1:
        rot = np.linalg.inv(np.hstack((q2_ras3,q1_ras3,q3_ras3)))
    else:
        rot = np.linalg.inv(np.hstack((q2_ras3,q3_ras3,q1_ras3)))
    
    close_enough = 1e-12
    if (np.linalg.det(rot) - 1.0) > close_enough:
        print(f"WARNING: det(rot) is {np.linalg.det(rot)}, should be 1.0", )
    
    trans = -1 * (rot @ p1_ras3)
    T_ras2ras = np.eye(4)
    T_ras2ras[0:3,0:3] = rot
    T_ras2ras[0:3,3] = trans.ravel()
    
    # See registered_image.py:133
    #    `T = np.dot(np.dot(flip, T), flip)`
    # Note that registered_image.py:133 only works becuase flip==inv(flip)
    # here we use the more general formula
    T_lps2lps = R2L @ T_ras2ras @ np.linalg.inv(R2L)
    
    return T_ras2ras, T_lps2lps

In [4]:
# Convert a 4x4 numpy matrix representing a transform to string
# To be used with the --trans flag of auto_register.py
def trans2str(trans):
    close_enough = 1e-12
    if (np.linalg.det(trans[0:3,0:3]) - 1.0) > close_enough:
        print(f"WARNING: det(trans[0:3,0:3]) is {np.linalg.det(trans[0:3,0:3])}, should be 1.0")

    trans_1d = T_l2l.ravel()
    trans_str = ''
    for i in trans_1d:
        trans_str += str(i) + ' '
    return trans_str

In [16]:
# Wrapper around calc_aa() for all possible dim/polarity combinations
def calc_all_aas(img_filename, p1_vox4, p2_vox4):
    dim_polarity = np.array([[0, 1],
                             [0,-1],
                             [1, 1],
                             [1,-1],
                             [2, 1],
                             [2,-1]], dtype=np.int32)
    
    for i in range(dim_polarity.shape[0]):
        dim=dim_polarity[i,0]
        pol=dim_polarity[i,1]
        print(f'Running with dim={dim} and pol={pol}:')
        T_r2r, T_l2l = calc_aa(img_filename, p1_vox, p2_vox, align_dim=dim, polarity=pol)
        T_rot = rotation.from_matrix(T_r2r[0:3,0:3])
        T_angle = np.linalg.norm(T_rot.as_rotvec(degrees=True))
        T_l2l_string = trans2str(T_l2l)
        print('T_r2r:')
        print(T_r2r)
        print('T_l2l:')
        print(T_l2l)
        print('T_l2l as a string (for `auto_register.py -trans`)')
        print(T_l2l_string)
        print(f'The angle of T is {T_angle} degrees')
        print('-------------------------------------------------------')

In [7]:
img_filename = '/home/paul/lcn/20230427-bay2-areg-v2-test/img-00000.nii.gz'

img = nb.load(img_filename)
vox2ras, grad2ras = calc_transforms(img)

# Using double brackets [[]] here so that the 1-d vector is transposable
#   - https://stackoverflow.com/a/10546291
# This is so silly, numpy.. It's bad enough that np.matrix got deprecated 
# but now you make us do this shit..

# tip of nose on head phantom
p1_vox = np.array([[3, 46, 31, 1]]).T

# gap at back of head phantom
p2_vox = np.array([[51, 38, 31, 1]]).T

p1_ras = vox2ras @ p1_vox
p2_ras = vox2ras @ p2_vox

## Validate

In [8]:
# Use freeview to verify ras/vox coords conversion
print("p1_vox:", p1_vox.T) 
print("p1_ras:", p1_ras.T)
print("p2_vox:", p2_vox.T) 
print("p2_ras:", p2_ras.T)

p1_vox: [[ 3 46 31  1]]
p1_ras: [[  2. 116. -56.   1.]]
p2_vox: [[51 38 31  1]]
p2_ras: [[  2. -76. -24.   1.]]


In [9]:
# Make p1_vox at origin; make p2_vox somewhere along x-axis (positive dir)
T_r2r, T_l2l = calc_aa(img_filename, p1_vox, p2_vox, align_dim=0)
# After applying T_r2r to p1_ras and p2_ras, we get:
#   - p1_ras_new
#   - p2_ras_new
p1_ras_new = T_r2r @ p1_ras
p2_ras_new = T_r2r @ p2_ras

print("p1_ras_new:", p1_ras_new.T)
print("p2_ras_new:", p2_ras_new.T)

close_enough = 1e-12
x_axis3 = np.array([[1.0,0.0,0.0]]).T
origin = np.array([[0.0,0.0,0.0,1.0]]).T

is_p1_ras_new_at_origin = np.all((p1_ras_new - origin) < close_enough)
# If 2 vectors are nearly parallel, their cross product is nearly zero
is_p2_ras_new_parallel_to_x_axis = np.linalg.norm(np.cross(p2_ras_new[0:3].T,x_axis3.T)) < close_enough

print("p1 is at origin?:   ", is_p1_ras_new_at_origin)
print("p2 is along x-axis?:", is_p2_ras_new_parallel_to_x_axis)

p1_ras_new: [[-5.32907052e-15  0.00000000e+00  7.10542736e-15  1.00000000e+00]]
p2_ras_new: [[ 1.94648401e+02  0.00000000e+00 -3.55271368e-15  1.00000000e+00]]
p1 is at origin?:    True
p2 is along x-axis?: True


In [10]:
# Make p1_vox at origin; make p2_vox somewhere along x-axis (negative dir)
T_r2r, T_l2l = calc_aa(img_filename, p1_vox, p2_vox, polarity=-1)

p1_ras_new = T_r2r @ p1_ras
p2_ras_new = T_r2r @ p2_ras

print("p1_ras_new:", p1_ras_new.T)
print("p2_ras_new:", p2_ras_new.T)

p1_ras_new: [[5.32907052e-15 0.00000000e+00 7.10542736e-15 1.00000000e+00]]
p2_ras_new: [[-1.94648401e+02  0.00000000e+00 -3.55271368e-15  1.00000000e+00]]


In [11]:
# Make p1_vox at origin; make p2_vox somewhere along y-axis (positive dir)
T_r2r, T_l2l = calc_aa(img_filename, p1_vox, p2_vox, align_dim=1)

p1_ras_new = T_r2r @ p1_ras
p2_ras_new = T_r2r @ p2_ras

print("p1_ras_new:", p1_ras_new.T)
print("p2_ras_new:", p2_ras_new.T)

p1_ras_new: [[ 0.00000000e+00 -5.32907052e-15  7.10542736e-15  1.00000000e+00]]
p2_ras_new: [[ 0.00000000e+00  1.94648401e+02 -3.55271368e-15  1.00000000e+00]]


In [12]:
# Make p1_vox at origin; make p2_vox somewhere along y-axis (negative dir)
T_r2r, T_l2l = calc_aa(img_filename, p1_vox, p2_vox, align_dim=1, polarity=-1)

p1_ras_new = T_r2r @ p1_ras
p2_ras_new = T_r2r @ p2_ras

print("p1_ras_new:", p1_ras_new.T)
print("p2_ras_new:", p2_ras_new.T)

p1_ras_new: [[0.00000000e+00 5.32907052e-15 7.10542736e-15 1.00000000e+00]]
p2_ras_new: [[ 0.00000000e+00 -1.94648401e+02 -3.55271368e-15  1.00000000e+00]]


In [13]:
# Make p1_vox at origin; make p2_vox somewhere along z-axis (positive dir)
T_r2r, T_l2l = calc_aa(img_filename, p1_vox, p2_vox, align_dim=2, polarity=1)

p1_ras_new = T_r2r @ p1_ras
p2_ras_new = T_r2r @ p2_ras

print("p1_ras_new:", p1_ras_new.T)
print("p2_ras_new:", p2_ras_new.T)

p1_ras_new: [[ 0.00000000e+00  7.10542736e-15 -1.77635684e-15  1.00000000e+00]]
p2_ras_new: [[7.77156117e-16 3.55271368e-15 1.94648401e+02 1.00000000e+00]]


In [14]:
# Make p1_vox at origin; make p2_vox somewhere along z-axis (negative dir)
T_r2r, T_l2l = calc_aa(img_filename, p1_vox, p2_vox, align_dim=2, polarity=-1)

p1_ras_new = T_r2r @ p1_ras
p2_ras_new = T_r2r @ p2_ras

print("p1_ras_new:", p1_ras_new.T)
print("p2_ras_new:", p2_ras_new.T)

p1_ras_new: [[0.00000000e+00 7.10542736e-15 1.77635684e-15 1.00000000e+00]]
p2_ras_new: [[-7.77156117e-16  3.55271368e-15 -1.94648401e+02  1.00000000e+00]]


In [17]:
calc_all_aas(img_filename,p1_vox,p2_vox)

Running with dim=0 and pol=1:
T_r2r:
[[-0.00000000e+00 -9.86393924e-01  1.64398987e-01  1.23628038e+02]
 [ 9.98474572e-01  9.07704156e-03  5.44622494e-02 -0.00000000e+00]
 [-5.52134883e-02  1.64148208e-01  9.84889251e-01  3.62230328e+01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
T_l2l:
[[ 0.00000000e+00 -9.86393924e-01 -1.64398987e-01 -1.23628038e+02]
 [ 9.98474572e-01  9.07704156e-03 -5.44622494e-02  0.00000000e+00]
 [ 5.52134883e-02 -1.64148208e-01  9.84889251e-01  3.62230328e+01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
T_l2l as a string (for `auto_register.py -trans`)
-0.9984745718892402 -0.009077041562629453 0.05446224937577673 4.440892098500626e-16 -0.05521348830312205 0.1641482084687412 -0.9848892508124474 -36.22303283972932 1.0681541315875595e-18 -0.9863939238321436 -0.16439898730535726 -123.62803845362866 0.0 0.0 0.0 1.0 
The angle of T is 90.17285325306398 degrees
-------------------------------------------------------
Runn